In [21]:
import requests
import os 
import json
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
def get_bar_data(api_keys, url, symbols, start, end, limit=1000, timeframe='15Min'):
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
        "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
    }
    
    params = {'symbols': symbols, 
          'timeframe':timeframe, 
          'start':start, 
          'end': end,
          'limit': limit}

    response = requests.get(url, headers=headers, params=params)

    data = response.json()

    df = pd.DataFrame(data['bars'][symbols])
    df.columns = ['close', 'high', 'low', 'trade_count', 'open', 'time', 'volume', 'volume_weighted_avg']
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time',inplace=True)
    df = df[['open', 'close', 'high', 'low', 'trade_count', 'volume', 'volume_weighted_avg']]

    return df

In [14]:
os.chdir('/Users/joemccann/MLProjects/alpaca_practice')

api_keys = json.load(open('apikey.json', 'r'))
url = "https://data.alpaca.markets/v2/stocks/bars?"
symbols = 'TSLA'
start = '2024-01-10'
end =  '2024-01-10'

In [26]:
df = get_bar_data(api_keys, url, symbols, start, end)

In [27]:
df

,open,close,high,low,trade_count,volume,volume_weighted_avg
time,,,,,,,
2024-01-10 00:00:00+00:00,234.4700,234.49,234.60,234.44,692,27008,234.529897
2024-01-10 00:15:00+00:00,234.4995,234.60,234.62,234.40,650,34546,234.503454
2024-01-10 00:30:00+00:00,234.5900,234.18,234.59,234.18,938,41683,234.275869
2024-01-10 00:45:00+00:00,234.2000,234.20,234.30,234.11,869,60657,234.187490
2024-01-10 09:00:00+00:00,235.5100,236.39,236.50,235.30,1248,44570,235.990301
...,...,...,...,...,...,...,...
2024-01-10 23:45:00+00:00,233.7400,233.74,233.80,233.66,353,12741,233.748357
2024-01-11 00:00:00+00:00,233.7600,233.72,233.76,233.66,452,10900,233.717723
2024-01-11 00:15:00+00:00,233.7000,233.66,233.74,233.66,422,19795,233.691105
